    # Setup:
    1. Create environment:
        In terminal, run:
        `conda env create -n lccd_test_env -f studio/app/optinist/wrappers/lccd/conda/lccd.yaml`

        `conda activate lccd_test_env`
    
    2. Install additional packages:
    
       `pip install pynwb imageio ipython jupyter notebook plotly "pydantic<2.0.0" python-dotenv uvicorn xmltodict bcrypt matplotlib "scikit-learn==1.1.*"`

      - If running in VS code, you may need to restart and/or select the correct environment with \"Python: Select Interpreter\",
    
    3. Run this notebook


In [6]:
import os
import sys
import uuid
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))

# Import OptiNiSt core data modules
from studio.app.dir_path import DIRPATH
from studio.app.common.dataclass import ImageData
from studio.app.optinist.dataclass import FluoData
# Import ROI detection modules
from studio.app.optinist.wrappers.lccd import lccd_detection
# Import OptiNiSt analysis modules
from studio.app.optinist.wrappers.optinist.dimension_reduction.tsne import TSNE

import numpy as np

# Import visualization modules
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Create input/output directories
input_dir = os.path.join(DIRPATH.INPUT_DIR, "1") 
os.makedirs(input_dir, exist_ok=True)
unique_id = str(uuid.uuid4())[:8]  # Generate 8-char unique ID

In [5]:
# Input file path
input_file = os.path.join(input_dir, "sample_mouse2p_image.tiff")
sample_data = ImageData([input_file])

In [8]:
# Set parameters for LCCD (Low Computational-cost Cell Detection)
lccd_detection_params = {
    'blob_detector': {
      'filtersize1': 100,
      'filtersize2': 4,
      'sigma': 1.25,
      'fsize': 30,
      'min_area': 20,
      'max_area': 50,
      'sparse': False
    },
    'roi_integration': {
      'overlap_threshold': 0.4,
      'min_area': 20,
      'max_area': 100,
      'sparse': False
    },
    'lccd': {
      'frame_divider': 100
    },
    'dff': {
      'f0_frames': 100,
      'f0_percentile': 8
    }
}

In [ ]:
# Create output directory for file conversion,
lccd_function_id = f"lccd_detect_{unique_id}"
lccd_output_dir = os.path.join(DIRPATH.OUTPUT_DIR, "1", unique_id, lccd_function_id)
os.makedirs(lccd_output_dir, exist_ok=True)

In [ ]:
# Perform LCCD detection
ret_lccd = lccd_detection.lccd_detect(sample_data, lccd_output_dir, lccd_detection_params)

In [ ]:
# Plot results of LCCD ROI detection 
# (no motion correction performed in this example)

# Extract data from results
fluorescence_data = ret_lccd['fluorescence'].data  # Shape: (50, 2000)
dff_data = ret_lccd['dff'].data  # Shape: (50, 2000)
iscell_data = ret_lccd['iscell'].data  # Shape: (50,)
roi_masks = ret_lccd['cell_roi'].data  # Shape: (128, 128)

# Calculate mean image from sample_data
mean_img = np.mean(sample_data.data, axis=0)  # Average across time

# Create subplot figure
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=('Mean Image', 'All ROI Masks',
                  'Mean Activity', 'Individual ROI Traces'),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Mean Image
fig.add_trace(
    go.Heatmap(
        z=mean_img,
        colorscale='gray', 
        showscale=False,
        name='Mean Image',
        showlegend=False
    ),
    row=1, col=1
)

# 2. ROI Masks
fig.add_trace(
    go.Heatmap(
        z=roi_masks,
        colorscale='viridis',
        showscale=True,
        name='ROI Masks',
        colorbar=dict(
            title='ROI #',
            len=0.4,
            y=0.8
        )
    ),
    row=1, col=2
)

# 3. Mean Activity
mean_activity = np.mean(fluorescence_data, axis=0)
time_points = np.arange(len(mean_activity))

fig.add_trace(
    go.Scatter(
        x=time_points,
        y=mean_activity,
        mode='lines',
        name='Mean Activity',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False,
        legendgroup='mean_activity',
        legendgrouptitle_text='Mean Activity'
    ),
    row=2, col=1
)

# 4. Individual ROI Traces
colors = px.colors.qualitative.Set3 * (1 + fluorescence_data.shape[0]//len(px.colors.qualitative.Set3))  # Repeat colors if needed
for i in range(fluorescence_data.shape[0]):  # Plot all ROIs
    color = colors[i % len(px.colors.qualitative.Set3)]
    fig.add_trace(
        go.Scatter(
            x=time_points,
            y=fluorescence_data[i,:],
            mode='lines',
            name=f'ROI {i+1}',
            line=dict(
                color=color,
                width=1
            ),
            opacity=0.7,
            showlegend=(i < 10),  # Only show first 10 traces in legend
            legendgroup='roi_traces',
            legendgrouptitle_text='ROI'
        ),
        row=2, col=2
    )

# Update layout
fig.update_layout(
    height=800,
    width=1000,
    title=dict(
        text="LCCD Analysis Results",
        x=0.5,
        y=0.95
    ),
    showlegend=True,
    legend=dict(
        x=1.0,
        y=0.1,
        traceorder='grouped',
        tracegroupgap=5
    )
)

# Update axes labels
fig.update_xaxes(title_text="X Position", row=1, col=1)
fig.update_yaxes(title_text="Y Position", row=1, col=1)
fig.update_xaxes(title_text="X Position", row=1, col=2)
fig.update_yaxes(title_text="Y Position", row=1, col=2)
fig.update_xaxes(title_text="Time Points", row=2, col=1)
fig.update_yaxes(title_text="Fluorescence", row=2, col=1)
fig.update_xaxes(title_text="Time Points", row=2, col=2)
fig.update_yaxes(title_text="Fluorescence", row=2, col=2)

fig.show()

In [35]:
# Set parameters for TSNE
tsne_params = {
    'transpose': True,
    'standard_mean': True,
    'standard_std': True,
    'TSNE': {
        'n_components': 2,
        'perplexity': 30.0,
        'early_exaggeration': 12.0,
        'learning_rate': 200.0,
        'n_iter': 1000,
        'n_iter_without_progress': 300,
        'min_grad_norm': 0.0000001,
        'metric': 'euclidean',
        'init': 'random',
        'random_state': 0,
        'method': 'barnes_hut',
        'angle': 0.5,
        'n_jobs': 1,
    }
}

In [30]:
# Create output directory for TSNE
tsne_function_id = f"tsne_{unique_id}"
tsne_output_dir = os.path.join(DIRPATH.OUTPUT_DIR, "1", unique_id, tsne_function_id)
os.makedirs(tsne_output_dir, exist_ok=True)


In [ ]:
# Run TSNE analysis
fluo_data = FluoData(ret_lccd['fluorescence'].data, file_name="fluorescence")

tsne_results = TSNE(fluo_data, tsne_output_dir, ret_lccd['iscell'], tsne_params)

# Print TSNE results contents
print("\nTSNE Results:")
for key, value in tsne_results.items():
    if key != 'nwbfile': 
        print(f"\n{key}:")
        print(f"Shape: {value.data.shape}")

In [ ]:
# Create visualization of TSNE results

# Create figure
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('TSNE Projection', 'TSNE Projection (Time-Coloured)'),
                    horizontal_spacing=0.15)

# Get TSNE projection data and transpose it
proj_data = tsne_results['projectedNd'].data.T  # Transpose to get (time_points, components)
time_points = np.arange(len(proj_data))

# 1. Basic TSNE plot
fig.add_trace(
    go.Scatter(
        x=proj_data[:, 0],
        y=proj_data[:, 1],
        mode='markers',
        marker=dict(
            size=3,
            color='blue',
            opacity=0.6
        ),
        name='TSNE Projection',
        showlegend=False
    ),
    row=1, col=1
)

# 2. TSNE trajectory plot colored by time
fig.add_trace(
    go.Scatter(
        x=proj_data[:, 0],
        y=proj_data[:, 1],
        mode='markers',
        marker=dict(
            size=3,
            color=time_points,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(
                title='Time Point',
                len=1,
                y=0.5
            )
        ),
        name='TSNE Projection (Time-Colored)',
        showlegend=False
    ),
    row=1, col=2
)

# Update axes labels
for i in [1, 2]:
    fig.update_xaxes(title_text="TSNE 1", row=1, col=i)
    fig.update_yaxes(title_text="TSNE 2", row=1, col=i)

# Update layout
fig.update_layout(
    height=500,
    width=1000,
    title=dict(
        text="TSNE Analysis of LCCD Results",
        x=0.5,
        y=0.95
    ),
    showlegend=False,
)

fig.show()